### Let's construct LeNet in Keras!

#### First let's load and prep our MNIST data

In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/ZeroPadding2D
# ZeroPadding2D This layer can add rows and columns of zeros at the top, bottom, left and right side of an image tensor

from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.datasets import mnist
from keras.utils import np_utils
import keras

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our data thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image 
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# Now we one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

### Now let's create our layers to replicate LeNet

In [3]:
# create model
model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(20, (5, 5),
                 padding = "same", 
                 input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

model.add(Conv2D(50, (5, 5),
                 padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(num_classes))
model.add(Activation("softmax"))
           
model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])
    
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 20)        520       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2450)              0

### Now let us train LeNet on our MNIST Dataset

In [4]:
# Training Parameters
batch_size = 128
epochs = 10


history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save("Trained Models/mnist_LeNet.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/10
469/469 [==============================] - 5s 7ms/step - loss: 2.2934 - accuracy: 0.1522 - val_loss: 2.2683 - val_accuracy: 0.2102
Epoch 2/10
469/469 [==============================] - 3s 6ms/step - loss: 2.2597 - accuracy: 0.2371 - val_loss: 2.2296 - val_accuracy: 0.3787
Epoch 3/10
469/469 [==============================] - 3s 6ms/step - loss: 2.2194 - accuracy: 0.4223 - val_loss: 2.1787 - val_accuracy: 0.5336
Epoch 4/10
469/469 [==============================] - 3s 6ms/step - loss: 2.1637 - accuracy: 0.5598 - val_loss: 2.1068 - val_accuracy: 0.6344
Epoch 5/10
469/469 [==============================] - 3s 6ms/step - loss: 2.0862 - accuracy: 0.6541 - val_loss: 2.0024 - val_accuracy: 0.6991
Epoch 6/10
469/469 [==============================] - 3s 6ms/step - loss: 1.9715 - accuracy: 0.7090 - val_loss: 1.8492 - val_accuracy: 0.7437
Epoch 7/10
469/469 [==============================] - 3s 6ms/step - loss: 1.8075 - accuracy: 0.7431 - val_loss: 1.6405 - val_accuracy: 0.7673
Epoch 